In [1]:
%load_ext autoreload
%autoreload 2

%pylab inline

import dataloaders
import models
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics 
import sklearn.cluster as cluster
import numpy as np 
import random
import classifiers
from torch.autograd import Variable
import torch
import torch.nn as nn 
import torch.optim as optim
import sklearn.model_selection as model_selection
import util
import json
from collections import defaultdict

from imblearn.under_sampling import RandomUnderSampler

Populating the interactive namespace from numpy and matplotlib


Using Theano backend.


In [3]:

dataset = 'wiki-Rfa'
input_file = f'../data/{dataset}-cleaned.csv'
delimiter = ' '
ratio = 0.8
data = dataloaders.UnsplitDataset(filepath=input_file, ratio=ratio, delimiter=delimiter)

In [4]:
(data.get_num_nodes()) 

11368

In [5]:
X, y = data.get_shuffled_data()

In [6]:
# parameters for PsuedoKernel 
num_nodes = data.get_num_nodes()
dims = 16
epochs = 100
lr = 0.15
lr_decay=0.0
weight_decay=0.0
lam = 0.00055
p = 2

In [7]:
model_fitter = models.fit_pseudo_kernel_model

In [8]:
num_splits = 5

In [9]:
kf = model_selection.KFold(n_splits=num_splits)

In [10]:
kf.get_n_splits(X)

5

In [11]:
delta = 1
delta0 = 0.5
dims_array = [dims, 20, 20]
frac1 = 1
frac0 = 1
p0 = True if frac0 > 0 else 0

In [12]:

reports_pksem = []
reports_sine = []
reports_hope = []
#operation = 'hadamard'
operations = ['hadamard', 'l1', 'l2', 'concat', 'average']
reports_pksem = defaultdict(list)
reports_sine = defaultdict(list)
reports_hope = defaultdict(list)
count = 1
for train_idx, test_idx in kf.split(X):
    
    print("======================================================================================")
    print(f"========================          Fold #{count}                    ========================")
    print("======================================================================================")
    
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    # train and evaluate pksem
    kernel_model = models.fit_pseudo_kernel_model(num_nodes, dims, X, y, epochs=epochs, p=p, 
                                              lr=lr,lr_decay=lr_decay, lam=lam, weight_decay=weight_decay, 
                                                  undersample=True)
    
    for operation in operations:
        clf_pksem = linear_model.LogisticRegression()
        report = classifiers.train_and_evaluate_classifier(clf_pksem, kernel_model, X_train, y_train, X_test, y_test,
                                                    operation=operation, undersample=True, ratio=2)
        reports_pksem[operation].append(report)
    
    # train and evaluate SiNE
    triples, triples0 = util.triples_from_array(X_train, y_train)
    batch_size = int(frac1 * len(triples))
    batch_size0 = int(frac0 * len(triples0))
    sine_model = models.fit_sine_model(num_nodes, dims_array, triples, triples0, delta, delta0,batch_size, batch_size0, epochs, lr=lr, lr_decay=lr_decay,lam=lam, p=p, p0=p0)
    for operation in operations:
        clf_sine = linear_model.LogisticRegression()
        report = classifiers.train_and_evaluate_classifier(clf_sine, sine_model, X_train, y_train, X_test, y_test,
                                                    operation=operation, undersample=True, ratio=2)
        reports_sine[operation].append(report)
        
    # train and evaluate hope
    hope_model = models.fit_hope(dims, X_train, num_nodes)
    for operation in operations:
        clf_hope = linear_model.LogisticRegression()
        report = classifiers.train_and_evaluate_classifier(clf_hope, hope_model, X_train, y_train, X_test, y_test,
                                                    operation=operation, undersample=True, ratio=2)
        reports_hope[operation].append(report)    
    count += 1
    
    
    
    
    

========================          Fold #1                    ========================
The loss at epoch  1  was  0.7019433975219727
The loss at epoch  2  was  0.727384090423584
The loss at epoch  3  was  0.6993259191513062
The loss at epoch  4  was  0.6865422129631042
The loss at epoch  5  was  0.6381019949913025
The loss at epoch  6  was  0.6378362774848938
The loss at epoch  7  was  0.6195421814918518
The loss at epoch  8  was  0.6007304191589355
The loss at epoch  9  was  0.5803611278533936
The loss at epoch  10  was  0.5511767864227295
The loss at epoch  11  was  0.5379834175109863
The loss at epoch  12  was  0.5260531902313232
The loss at epoch  13  was  0.5119312405586243
The loss at epoch  14  was  0.5010831356048584
The loss at epoch  15  was  0.5092520117759705
The loss at epoch  16  was  0.5459374785423279
The loss at epoch  17  was  0.5303258895874023
The loss at epoch  18  was  0.4972645342350006
The loss at epoch  19  was  0.4856754243373871
The loss at epoch  20  was  0.4

Loss at epoch  66  is  0.19147352874279022
Loss at epoch  67  is  0.18581929802894592
Loss at epoch  68  is  0.19234199821949005
Loss at epoch  69  is  0.19820213317871094
Loss at epoch  70  is  0.19766324758529663
Loss at epoch  71  is  0.18230067193508148
Loss at epoch  72  is  0.17781387269496918
Loss at epoch  73  is  0.17750434577465057
Loss at epoch  74  is  0.1836603283882141
Loss at epoch  75  is  0.1912170946598053
Loss at epoch  76  is  0.19887951016426086
Loss at epoch  77  is  0.18327569961547852
Loss at epoch  78  is  0.17874623835086823
Loss at epoch  79  is  0.1785714477300644
Loss at epoch  80  is  0.17471927404403687
Loss at epoch  81  is  0.17314304411411285
Loss at epoch  82  is  0.17675895988941193
Loss at epoch  83  is  0.19365251064300537
Loss at epoch  84  is  0.21500137448310852
Loss at epoch  85  is  0.1928710639476776
Loss at epoch  86  is  0.18134842813014984
Loss at epoch  87  is  0.17723174393177032
Loss at epoch  88  is  0.17317548394203186
Loss at epoch  

/Users/inzamamrahaman/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/inzamamrahaman/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
========================          Fold #2                    ========================
The loss at epoch  1  was  0.6910271644592285
The loss at epoch  2  was  0.7213472127914429
The loss at epoch  3  was  0.680811882019043
The loss at epoch  4  was  0.6551201939582825
The loss at epoch  5  was  0.6516225337982178
The loss at epoch  6  was  0.6332229375839233
The loss at epoch  7  was  0.6124728918075562
The loss at epoch  8  was  0.5990285873413086
The loss at epoch  9  was  0.5817000865936279
The loss at epoch  10  was  0.5665865540504456
The loss at epoch  11  was  0.5679658651351929
The loss at epoch  12  was  0.5

Loss at epoch  58  is  0.2696937024593353
Loss at epoch  59  is  0.2653396427631378
Loss at epoch  60  is  0.2566571533679962
Loss at epoch  61  is  0.25163331627845764
Loss at epoch  62  is  0.2452801764011383
Loss at epoch  63  is  0.24193640053272247
Loss at epoch  64  is  0.23605965077877045
Loss at epoch  65  is  0.23347455263137817
Loss at epoch  66  is  0.22582976520061493
Loss at epoch  67  is  0.2193928211927414
Loss at epoch  68  is  0.21868914365768433
Loss at epoch  69  is  0.22222010791301727
Loss at epoch  70  is  0.22331799566745758
Loss at epoch  71  is  0.21686413884162903
Loss at epoch  72  is  0.21394747495651245
Loss at epoch  73  is  0.2125861942768097
Loss at epoch  74  is  0.21463420987129211
Loss at epoch  75  is  0.20781733095645905
Loss at epoch  76  is  0.20325212180614471
Loss at epoch  77  is  0.2005568891763687
Loss at epoch  78  is  0.20388898253440857
Loss at epoch  79  is  0.20753632485866547
Loss at epoch  80  is  0.20107658207416534
Loss at epoch  81 

The loss at epoch  96  was  0.45327693223953247
The loss at epoch  97  was  0.45385801792144775
The loss at epoch  98  was  0.4536881744861603
The loss at epoch  99  was  0.4515547454357147
The loss at epoch  100  was  0.45097875595092773
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Loss at epoch  1  is  1.00444757938385
Loss at epoch  2  is  0.9114053845405579
Loss at epoch  3  is  0.28934451937675476
Loss at epoch  4  is  0.27682289481163025
Loss at epoch  5  is  0.27186647057533264
Loss at epoch  6  is  0.3809698820114136
Loss at epoch  7  is  0.30639252066612244
Lo

The loss at epoch  30  was  0.4712774455547333
The loss at epoch  31  was  0.4698914587497711
The loss at epoch  32  was  0.468100905418396
The loss at epoch  33  was  0.470586895942688
The loss at epoch  34  was  0.46759429574012756
The loss at epoch  35  was  0.4695945382118225
The loss at epoch  36  was  0.46822673082351685
The loss at epoch  37  was  0.4681515097618103
The loss at epoch  38  was  0.4678119122982025
The loss at epoch  39  was  0.4666503965854645
The loss at epoch  40  was  0.4658638536930084
The loss at epoch  41  was  0.4664618670940399
The loss at epoch  42  was  0.4646475911140442
The loss at epoch  43  was  0.46519455313682556
The loss at epoch  44  was  0.46375107765197754
The loss at epoch  45  was  0.464921772480011
The loss at epoch  46  was  0.4628133773803711
The loss at epoch  47  was  0.4639939069747925
The loss at epoch  48  was  0.4626053273677826
The loss at epoch  49  was  0.4638056755065918
The loss at epoch  50  was  0.4605104923248291
The loss at 

Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
Assembling training set features....
Fitting classifier model
Assembling testing set features
SVD error (low rank): 3.450095
[[ -4.97665242e-21   7.31200395e-21   2.51661355e-20 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -1.64937337e-03   6.92053142e-04  -7.71097492e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -2.04340941e-03   1.73476334e-03  -8.87759649e-04 ...,  -2.36059312e-02
   -1.86398262e-02   2.38798001e-02]
 ..., 
 [  1.45666649e-19  -1.86370783e-19  -4.61301435e-19 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -1.11812975e-19   1.66508356e-19   3.59833331e-19 ...,  -1.61937871e-05
    1.58433369e-05   1.84196175e-05]
 [  4.71284489e-06   6.91899013e-06   3.07509925e-06 ...,   0.0

Loss at epoch  30  is  0.23132088780403137
Loss at epoch  31  is  0.21925996243953705
Loss at epoch  32  is  0.20731596648693085
Loss at epoch  33  is  0.20569458603858948
Loss at epoch  34  is  0.23327337205410004
Loss at epoch  35  is  0.22639226913452148
Loss at epoch  36  is  0.2173156887292862
Loss at epoch  37  is  0.20172376930713654
Loss at epoch  38  is  0.20045596361160278
Loss at epoch  39  is  0.19791755080223083
Loss at epoch  40  is  0.21357570588588715
Loss at epoch  41  is  0.21690797805786133
Loss at epoch  42  is  0.20101025700569153
Loss at epoch  43  is  0.18903928995132446
Loss at epoch  44  is  0.19013087451457977
Loss at epoch  45  is  0.1885724663734436
Loss at epoch  46  is  0.21397387981414795
Loss at epoch  47  is  0.22881531715393066
Loss at epoch  48  is  0.2133147418498993
Loss at epoch  49  is  0.2002967745065689
Loss at epoch  50  is  0.19486401975154877
Loss at epoch  51  is  0.1915423721075058
Loss at epoch  52  is  0.19505317509174347
Loss at epoch  5

In [18]:

def average_scores(reports, scores=['auc', 'average_percision_score', 'macro_f1', 'micro_f1', 
                                   'kappa', 'mathew']):
    score_lst = dict()
    for key in scores:
        score_lst[key] = []
    for report in reports:
        for key in scores:
            score_lst[key].append(report[key])
    
    score_avg = dict()
    score_std = dict()
    for key in scores:
        score_avg[key] = np.mean(score_lst[key])
        score_std[key] = np.std(score_lst[key])
        
    return score_avg, score_std
print(average_scores(reports_sine['hadamard']))
print(average_scores(reports_pksem['hadamard']))
print(average_scores(reports_hope['hadamard']))

({'auc': 0.62427481030308407, 'average_percision_score': 0.84198364733386755, 'macro_f1': 0.53601587280117291, 'micro_f1': 0.78058148922651438, 'kappa': 0.12551380295156772, 'mathew': 0.17027745432571353}, {'auc': 0.017084549135235747, 'average_percision_score': 0.0090613645198908896, 'macro_f1': 0.012192315601670452, 'micro_f1': 0.00098955013926293892, 'kappa': 0.016619407077147586, 'mathew': 0.012920045313831693})
({'auc': 0.81849384637421019, 'average_percision_score': 0.93730708143671426, 'macro_f1': 0.70126513425261872, 'micro_f1': 0.79133422609800352, 'kappa': 0.40261846500973392, 'mathew': 0.40281186078181441}, {'auc': 0.010785268016943378, 'average_percision_score': 0.00449415725992051, 'macro_f1': 0.012950951813289552, 'micro_f1': 0.0070777905903845863, 'kappa': 0.026017673773063636, 'mathew': 0.026284188890030771})
({'auc': 0.59452542764662641, 'average_percision_score': 0.82588122572464262, 'macro_f1': 0.43762674327601436, 'micro_f1': 0.77817881100505681, 'kappa': 0.0, 'math

In [14]:
def average_scores(reports, scores=['auc', 'average_percision_score', 'macro_f1', 'micro_f1', 
                                   'kappa', 'mathew']):
    score_lst = dict()
    for key in scores:
        score_lst[key] = []
    for report in reports:
        for key in scores:
            score_lst[key].append(report[key])
    
    score_avg = dict()
    score_std = dict()
    for key in scores:
        score_avg[key] = np.mean(score_lst[key])
        score_std[key] = np.std(score_lst[key])
        
    return score_avg, score_std
    
    

In [15]:
result_link_pred_file = f'link-predv3?{dataset}.txt'

In [16]:
with open(result_link_pred_file, 'a') as fp:
    for operation in operations:
        r1 = average_scores(reports_pksem[operation])
        r2 = average_scores(reports_sine[operation])
        r3 = average_scores(reports_hope[operation])
        
        line = '{0},{1},{2},{3},{4},{5},{6},{7}\n\n'.format(input_file, 
                                                            operation, r1[0], r1[1], 
                                                            r2[0], r2[1],
                                                            r3[0], r3[1])
        fp.write(line)

In [17]:
abs(0.923 - 0.930)/0.923 * 100

0.758396533044421

In [19]:
np.log(38)

3.6375861597263857

In [20]:
np.log(1.57)

0.45107561936021673